# SVM Kernel Tricks Deep Dive

## Learning Objectives
- Understand the mathematics behind the kernel trick
- Visualize kernel-induced feature spaces
- Compare kernel performance on various datasets
- Implement a custom kernel function
- Analyze the kernel (Gram) matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel, linear_kernel

np.random.seed(42)
plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. Why Kernels? — The Feature Map Idea

A kernel function computes the inner product in a high-dimensional feature space **without explicitly mapping the data**:

$$K(x, z) = \langle \phi(x), \phi(z) \rangle$$

This is the **kernel trick** — it avoids the computational cost of working in the (possibly infinite-dimensional) feature space.

In [ ]:
# Demonstrate: non-linearly separable data becomes separable in higher dimensions
X_circles, y_circles = make_circles(n_samples=300, noise=0.05, factor=0.4, random_state=42)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Original 2D space — not linearly separable
axes[0].scatter(X_circles[y_circles==0, 0], X_circles[y_circles==0, 1], c='blue', label='Class 0', alpha=0.6)
axes[0].scatter(X_circles[y_circles==1, 0], X_circles[y_circles==1, 1], c='red', label='Class 1', alpha=0.6)
axes[0].set_title('Original 2D Space\n(Not linearly separable)')
axes[0].legend()
axes[0].set_aspect('equal')

# Why: This explicit mapping phi(x1,x2) = (x1^2, sqrt(2)*x1*x2, x2^2) shows
# what the polynomial kernel computes implicitly. In 3D, the circles become
# linearly separable — the kernel trick does this without materializing the map.
X_mapped = np.column_stack([
    X_circles[:, 0]**2,
    np.sqrt(2) * X_circles[:, 0] * X_circles[:, 1],
    X_circles[:, 1]**2
])

# 3D feature space
ax3d = fig.add_subplot(132, projection='3d')
ax3d.scatter(X_mapped[y_circles==0, 0], X_mapped[y_circles==0, 1], X_mapped[y_circles==0, 2], c='blue', alpha=0.4)
ax3d.scatter(X_mapped[y_circles==1, 0], X_mapped[y_circles==1, 1], X_mapped[y_circles==1, 2], c='red', alpha=0.4)
ax3d.set_title('Mapped to 3D\n(Linearly separable!)')
ax3d.set_xlabel('x1²')
ax3d.set_ylabel('√2·x1·x2')
ax3d.set_zlabel('x2²')
axes[1].set_visible(False)

# SVM with RBF kernel (does this implicitly)
clf = SVC(kernel='rbf', gamma='scale')
clf.fit(X_circles, y_circles)
xx, yy = np.meshgrid(np.linspace(-1.5, 1.5, 200), np.linspace(-1.5, 1.5, 200))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
axes[2].contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
axes[2].scatter(X_circles[:, 0], X_circles[:, 1], c=y_circles, cmap='coolwarm', edgecolors='k', s=30)
axes[2].set_title(f'RBF Kernel SVM\n(Acc: {clf.score(X_circles, y_circles):.3f})')
axes[2].set_aspect('equal')

plt.tight_layout()
plt.show()

## 2. Kernel Functions Compared

| Kernel | Formula | Feature Space | Use Case |
|--------|---------|---------------|----------|
| Linear | $K(x,z) = x^T z$ | Same as input | Linearly separable |
| Polynomial | $K(x,z) = (\gamma x^T z + r)^d$ | Finite dim | Polynomial boundaries |
| RBF (Gaussian) | $K(x,z) = \exp(-\gamma \|x-z\|^2)$ | Infinite dim | General non-linear |
| Sigmoid | $K(x,z) = \tanh(\gamma x^T z + r)$ | — | Neural network analog |

In [ ]:
# Compare all kernels on three different datasets
datasets = [
    ('Linearly separable', make_classification(n_samples=200, n_features=2, n_redundant=0, n_informative=2, n_clusters_per_class=1, random_state=42)),
    ('Moons', make_moons(n_samples=200, noise=0.15, random_state=42)),
    ('Circles', make_circles(n_samples=200, noise=0.1, factor=0.4, random_state=42)),
]

kernels = [
    ('Linear', {'kernel': 'linear'}),
    ('Poly (d=3)', {'kernel': 'poly', 'degree': 3, 'gamma': 'scale'}),
    ('RBF', {'kernel': 'rbf', 'gamma': 'scale'}),
    ('Sigmoid', {'kernel': 'sigmoid', 'gamma': 'scale'}),
]

fig, axes = plt.subplots(3, 4, figsize=(20, 15))

for row, (ds_name, (X, y)) in enumerate(datasets):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    for col, (k_name, k_params) in enumerate(kernels):
        ax = axes[row, col]
        clf = SVC(**k_params, C=1.0)
        clf.fit(X, y)
        score = clf.score(X, y)
        
        xx, yy = np.meshgrid(np.linspace(X[:,0].min()-0.5, X[:,0].max()+0.5, 150),
                             np.linspace(X[:,1].min()-0.5, X[:,1].max()+0.5, 150))
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
        ax.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
        ax.scatter(X[:,0], X[:,1], c=y, cmap='coolwarm', edgecolors='k', s=20)
        ax.set_title(f'{k_name}\nAcc: {score:.3f}, SVs: {len(clf.support_vectors_)}')
        if col == 0:
            ax.set_ylabel(ds_name, fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## 3. Kernel (Gram) Matrix Visualization

The Gram matrix $K_{ij} = K(x_i, x_j)$ captures pairwise similarities between all data points. A valid kernel must produce a positive semi-definite Gram matrix (Mercer's condition).

In [ ]:
# Generate data and compute kernel matrices
X, y = make_moons(n_samples=100, noise=0.1, random_state=42)
X = StandardScaler().fit_transform(X)

# Sort by class for block structure visualization
order = np.argsort(y)
X_sorted = X[order]
y_sorted = y[order]

# Compute kernel matrices
K_linear = linear_kernel(X_sorted)
K_poly = polynomial_kernel(X_sorted, degree=3, gamma=1, coef0=1)
K_rbf_low = rbf_kernel(X_sorted, gamma=0.1)
K_rbf_high = rbf_kernel(X_sorted, gamma=10)

fig, axes = plt.subplots(1, 4, figsize=(20, 5))
matrices = [
    ('Linear', K_linear),
    ('Poly (d=3)', K_poly),
    ('RBF (gamma=0.1)', K_rbf_low),
    ('RBF (gamma=10)', K_rbf_high),
]

for ax, (name, K) in zip(axes, matrices):
    im = ax.imshow(K, cmap='viridis', aspect='auto')
    ax.set_title(f'{name}\nrank={np.linalg.matrix_rank(K)}')
    ax.axhline(y=np.sum(y_sorted==0)-0.5, color='red', linewidth=1, linestyle='--')
    ax.axvline(x=np.sum(y_sorted==0)-0.5, color='red', linewidth=1, linestyle='--')
    plt.colorbar(im, ax=ax, shrink=0.8)

plt.suptitle('Kernel (Gram) Matrices — Red lines separate classes', fontsize=14)
plt.tight_layout()
plt.show()

# Why: A valid kernel (Mercer's condition) must produce a PSD Gram matrix.
# If min eigenvalue < 0, the "kernel" is invalid and SVM optimization may fail.
for name, K in matrices:
    eigenvalues = np.linalg.eigvalsh(K)
    is_psd = np.all(eigenvalues >= -1e-10)
    print(f'{name}: PSD={is_psd}, min eigenvalue={eigenvalues.min():.6f}')

## 4. RBF Gamma — Effect on Decision Boundary

The gamma parameter controls the "reach" of each support vector:

$$K(x, z) = \exp(-\gamma \|x - z\|^2)$$

- **Small gamma**: each point has wide influence → smooth boundary
- **Large gamma**: each point has narrow influence → complex boundary

In [ ]:
X, y = make_moons(n_samples=200, noise=0.2, random_state=42)
X = StandardScaler().fit_transform(X)

gammas = [0.01, 0.1, 1, 10, 100]
fig, axes = plt.subplots(1, 5, figsize=(25, 5))

for ax, gamma in zip(axes, gammas):
    clf = SVC(kernel='rbf', gamma=gamma, C=1)
    clf.fit(X, y)
    cv_score = cross_val_score(clf, X, y, cv=5).mean()
    
    xx, yy = np.meshgrid(np.linspace(-3, 3, 200), np.linspace(-3, 3, 200))
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    
    ax.contourf(xx, yy, Z, levels=20, cmap='coolwarm', alpha=0.5)
    ax.contour(xx, yy, Z, levels=[0], colors='black', linewidths=2)
    ax.scatter(X[:,0], X[:,1], c=y, cmap='coolwarm', edgecolors='k', s=30)
    ax.scatter(clf.support_vectors_[:,0], clf.support_vectors_[:,1],
               s=80, facecolors='none', edgecolors='lime', linewidths=1.5)
    ax.set_title(f'gamma={gamma}\nSVs={len(clf.support_vectors_)}\nCV={cv_score:.3f}')
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)

plt.suptitle('RBF Kernel: Effect of gamma on Decision Boundary', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Polynomial Kernel — Degree Effect

$$K(x, z) = (\gamma x^T z + r)^d$$

The degree $d$ controls the complexity of the polynomial boundary.

In [ ]:
X, y = make_moons(n_samples=200, noise=0.15, random_state=42)
X = StandardScaler().fit_transform(X)

degrees = [1, 2, 3, 5, 10]
fig, axes = plt.subplots(1, 5, figsize=(25, 5))

for ax, degree in zip(axes, degrees):
    clf = SVC(kernel='poly', degree=degree, gamma='scale', coef0=1, C=1)
    clf.fit(X, y)
    cv_score = cross_val_score(clf, X, y, cv=5).mean()
    
    xx, yy = np.meshgrid(np.linspace(-3, 3, 200), np.linspace(-3, 3, 200))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    ax.scatter(X[:,0], X[:,1], c=y, cmap='coolwarm', edgecolors='k', s=30)
    ax.set_title(f'Poly d={degree}\nSVs={len(clf.support_vectors_)}\nCV={cv_score:.3f}')

plt.suptitle('Polynomial Kernel: Effect of Degree', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Custom Kernel Function

sklearn allows custom kernels via `kernel='precomputed'` or a callable. A valid kernel must be symmetric and positive semi-definite.

In [ ]:
# Why: The Laplacian kernel uses L1 (Manhattan) distance instead of L2 (Euclidean).
# L1 is more robust to outliers in individual features. kernel='precomputed'
# tells SVM to use a pre-built Gram matrix instead of computing kernels internally.
def laplacian_kernel(X, Y, gamma=1.0):
    """Laplacian kernel using L1 (Manhattan) distance."""
    from scipy.spatial.distance import cdist
    dists = cdist(X, Y, metric='cityblock')  # L1 distance
    return np.exp(-gamma * dists)

# Compare standard RBF vs Laplacian on moons data
X, y = make_moons(n_samples=200, noise=0.15, random_state=42)
X = StandardScaler().fit_transform(X)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# RBF kernel
clf_rbf = SVC(kernel='rbf', gamma=1.0, C=1)
clf_rbf.fit(X, y)

# Laplacian kernel (precomputed)
K_train = laplacian_kernel(X, X, gamma=1.0)
clf_lap = SVC(kernel='precomputed', C=1)
clf_lap.fit(K_train, y)

# Plot decision boundaries
xx, yy = np.meshgrid(np.linspace(-3, 3, 150), np.linspace(-3, 3, 150))
grid = np.c_[xx.ravel(), yy.ravel()]

# RBF
Z_rbf = clf_rbf.predict(grid).reshape(xx.shape)
axes[0].contourf(xx, yy, Z_rbf, alpha=0.3, cmap='coolwarm')
axes[0].scatter(X[:,0], X[:,1], c=y, cmap='coolwarm', edgecolors='k', s=30)
axes[0].set_title(f'RBF Kernel (L2)\nAcc: {clf_rbf.score(X, y):.3f}')

# Laplacian
K_grid = laplacian_kernel(grid, X, gamma=1.0)
Z_lap = clf_lap.predict(K_grid).reshape(xx.shape)
axes[1].contourf(xx, yy, Z_lap, alpha=0.3, cmap='coolwarm')
axes[1].scatter(X[:,0], X[:,1], c=y, cmap='coolwarm', edgecolors='k', s=30)
axes[1].set_title(f'Laplacian Kernel (L1)\nAcc: {clf_lap.predict(K_train).mean():.3f}')

# Kernel matrices comparison
K_rbf_mat = rbf_kernel(X[:50], gamma=1.0)
K_lap_mat = laplacian_kernel(X[:50], X[:50], gamma=1.0)
axes[2].imshow(K_rbf_mat - K_lap_mat, cmap='RdBu', aspect='auto')
axes[2].set_title('RBF - Laplacian\n(Kernel matrix difference)')
plt.colorbar(axes[2].images[0], ax=axes[2])

plt.tight_layout()
plt.show()

## 7. Kernel Selection Guide

Practical guidelines for choosing the right kernel:

| Scenario | Recommended Kernel | Why |
|----------|-------------------|-----|
| n_features >> n_samples | Linear | High-D data is often linearly separable |
| n_samples >> n_features | RBF | Flexible, works well in low-D |
| Text classification (TF-IDF) | Linear | Sparse, high-dimensional |
| Image classification | RBF or Chi-squared | Non-linear patterns |
| Known polynomial relationship | Polynomial | Matches data structure |
| Default choice | RBF | Most versatile |

### Key Takeaways

1. **Kernel trick** avoids explicit high-dimensional mapping
2. **Gram matrix** encodes all pairwise similarities
3. **RBF** is the default choice — works well on most datasets
4. **gamma** controls overfitting: high gamma = complex boundary
5. **Custom kernels** must be symmetric and positive semi-definite
6. Always **scale features** before applying SVM with any kernel